Get OSM segments as specified in this [link](https://www.npmjs.com/package/movement-data-toolkit)

In [1]:
from shapely.geometry import Polygon, Point, LineString
from shapely.ops import cascaded_union
from shapely import wkt
import pandas as pd
import keplergl as kp
import utm
import geopandas as gpd
from itertools import compress 

#!pip install -i https://test.pypi.org/simple/ gtfs-functions-VERSION-1
import gtfs_functions as gtfs

# GTFS

We will start only a couple of lines in the city of Miami.

In [2]:
# Extract the GTFS
gtfs_path = r"C:\Users\santi\Dropbox (Remix)\Remix\Data Team\Data Analytics\Load\GTFS\MIAMI-DADE\MIAMI.zip"

routes, stops, stop_times, trips, shapes, gtfs_list = gtfs.import_gtfs(gtfs_path, busiest_date = False)

In [3]:
rfilter = routes.loc[routes.route_short_name.isin(['9', '10']), 'route_id'].unique()

sfilter = stop_times.loc[stop_times.route_id.isin(rfilter)].reset_index()

stops_filter = stops.loc[stops.stop_id.isin(sfilter.stop_id.unique())].reset_index()
shapes_filter = shapes.loc[shapes.shape_id.isin(sfilter.shape_id.unique())].reset_index()

# Cut the segments
bsegments_gdf = gtfs.cut_gtfs(stop_times, stops_filter, shapes_filter)
bsegments_gdf.route_id.unique()

In [920]:
m = kp.KeplerGl(height=400, data=dict(data=bsegments_gdf, name='Lines'))
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':    route_id  direction_id  stop_sequence start_stop_id  \
0     22753             0    …

In [23]:
# Buffer the lines 5 meters
b_segments = bsegments_gdf.loc[:,['segment_id', 'geometry']].drop_duplicates(subset='segment_id').reset_index().drop('index', axis=1)

# Save memory changing the data types
stop_segments = pd.DataFrame()
stop_segments['stops_id'] = b_segments.segment_id
stop_segments['segment_id'] = list(range(1, len(b_segments)+1))

b_segments['segment_id'] = stop_segments['segment_id']

int_col = b_segments.select_dtypes(include=['int64'])
columns = int_col.columns
converted_int = int_col.apply(pd.to_numeric, downcast='unsigned')
b_segments = pd.merge(b_segments.drop(columns, axis=1), converted_int, left_index=True, right_index=True, how='left')

# Get a coord system in meters
lat = b_segments.geometry[0].coords[0][1]
lon = b_segments.geometry[0].coords[0][0]

zone = utm.from_latlon(lat, lon)

def code(zone):
    #The EPSG code is 32600+zone for positive latitudes and 32700+zone for negatives.
    if lat <0:
        epsg_code = 32700 + zone[2]
    else:
        epsg_code = 32600 + zone[2]
    return epsg_code

# Create the buffers
buffer_size = 10

buffers = gpd.GeoDataFrame(data = b_segments['segment_id'], geometry = b_segments.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
kp.KeplerGl(height=400, data=dict(data=buffers, name='buffers'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':      segment_id                                           geometry
0             1  POL…

In [6]:
# Merge buffers into one polygon to speed up the spatial join (for filtering)
polygons = [p for p in buffers.geometry]
    
u = cascaded_union(polygons)

df = pd.DataFrame({'poly': [1], 'geometry':u.wkt})
df['geometry'] = df['geometry'].apply(wkt.loads)
lines_poly = gpd.GeoDataFrame(data=df.poly, geometry = df.geometry)

# Download OSM segments

In [7]:
# Query OSM with Overpass API
# Filter only roads (no POIs or nodes)
import requests
import json

bounds = lines_poly.geometry.total_bounds

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
(way["highway"~"motorway|trunk|primary|secondary|tertiary|unclassified|residential|service|living_street"]
["access"!~"private|no"]
({0}, {1}, {2}, {3}););
out geom;
""".format(bounds[1], bounds[0], bounds[3], bounds[2])

response = requests.get(overpass_url, 
                        params={'data': overpass_query})

data = response.json()
ways_df = pd.DataFrame(data['elements'])

In [8]:
# Transform into a geo data frame with line segments from node to node
way_ids = []
oneway = []
node_ids = []
lat_lon = []
segment_seq = []

for e in data['elements']:
    way_ids = way_ids + [e['id']]*len(e['nodes'])
    oneway = oneway + [e['tags'].get('oneway', '0')]*len(e['nodes'])
    segment_seq = segment_seq + list(range(1, len(e['nodes'])+1))
    node_ids = node_ids + e['nodes']
    lat_lon = lat_lon + e['geometry']

# Convert to int to save memory
oneway = [1 if s=='yes' else s for s in oneway] 
oneway = [0 if s=='no' else s for s in oneway] 
oneway = list(map(int, oneway))

# Parse the json into a dataframe
segments = pd.DataFrame()
segments['way_id'] = way_ids
segments['oneway'] = oneway
segments['segment_seq'] = segment_seq


# Get lat,lon values right
lat = [p['lat'] for p in lat_lon]
lon = [p['lon'] for p in lat_lon]

# Define our lists
bool_list = segments['way_id'] == segments['way_id'].shift(-1)
segment_nodes = ['{0} - {1}'.format(str(node_ids[i]), str(node_ids[i+1])) for i in range(0,len(node_ids)-1)]
segment_ids = list(range(1, len(segment_nodes)+1))
points =  [Point(lon[i], lat[i]) for i in range(0, len(lat))]
points_next = points[1:] + [None]

# Remove the last node of the segment (it is already in the last segment)
segment_ids = list(compress(segment_ids, bool_list)) 
segment_nodes = list(compress(segment_nodes, bool_list)) 
points = list(compress(points, bool_list)) 
points_next = list(compress(points_next, bool_list)) 
geometry = [LineString([points[i], points_next[i]]) for i in range(0,len(segment_ids))]

# Store the nodes of each segment
nodes_seg = pd.DataFrame({'osm_segment_id': segment_ids, 'nodes': segment_nodes})

# Keep the good data and create the geo data frame
segments = segments.loc[bool_list]
segments['osm_segment_id'] = segment_ids
segments_gdf = gpd.GeoDataFrame(data=segments, geometry = geometry)

In [9]:
# Save some memory
int_col = segments_gdf.select_dtypes(include=['int64'])
columns = int_col.columns
converted_int = int_col.apply(pd.to_numeric, downcast='unsigned')

segments_gdf = pd.merge(segments_gdf.drop(columns, axis=1), converted_int, left_index=True, right_index=True, how='left')
segments_gdf.info(memory_usage="deep")

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 51408 entries, 0 to 63643
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   geometry        51408 non-null  geometry
 1   way_id          51408 non-null  uint32  
 2   oneway          51408 non-null  int64   
 3   segment_seq     51408 non-null  uint8   
 4   osm_segment_id  51408 non-null  uint16  
dtypes: geometry(1), int64(1), uint16(1), uint32(1), uint8(1)
memory usage: 4.0 MB


In [10]:
%%time
# Keep only the osm segments that intersect the bus lines
osm = gpd.sjoin(segments_gdf, lines_poly, op='intersects').reset_index().drop(['index_right', 'poly', 'index'], axis=1)

Wall time: 20.8 s


In [874]:
osm.info(memory_usage="deep")

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3102 entries, 0 to 3101
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   geometry        3102 non-null   geometry
 1   way_id          3102 non-null   uint32  
 2   oneway          3102 non-null   int64   
 3   segment_seq     3102 non-null   uint8   
 4   osm_segment_id  3102 non-null   uint16  
dtypes: geometry(1), int64(1), uint16(1), uint32(1), uint8(1)
memory usage: 69.8 KB


In [11]:
m = kp.KeplerGl(data=dict(data=osm, name='segments'))
m.add_data(data=lines_poly, name='lines_poly')
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':                                                geometry     way_id  oneway  \
0     LIN…

# Join GTFS segments to OSM segments

## 1. Approach with overlay and area calculation (discarded, go to #2)

In [575]:
# Right now we filtered  OSM segments that intersect with lines_poly
# We will keep only those that more than 50% of their area falls within lines_poly
buffer_size = 5

buffers_10 = gpd.GeoDataFrame(data = b_segments['segment_id'], geometry = b_segments.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
polygons = [p for p in buffers_10.geometry]
u = cascaded_union(polygons)

df = pd.DataFrame({'poly': [1], 'geometry':u.wkt})
df['geometry'] = df['geometry'].apply(wkt.loads)
lines_poly_10 = gpd.GeoDataFrame(data=df.poly, geometry = df.geometry)


osm.crs = {'init':'epsg:4326'}
osm_bu = gpd.GeoDataFrame(data = osm.drop('geometry', axis=1), geometry = osm.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
osm_bu['area_osm'] = osm_bu.area

In [576]:
# %%time
# #osm_test = intersections.loc[intersections.segment_id=='8908-8826']
# segments_gdf.crs = {'init':'epsg:4326'}
# osm_bu = gpd.GeoDataFrame(data = segments_gdf.drop('geometry', axis=1), geometry = segments_gdf.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
# osm_bu['area_osm'] = osm_bu.area

last = gpd.overlay(osm_bu, lines_poly_10, how='intersection')
last['area_overlay'] = last.area
last['per'] = last['area_overlay']/last['area_osm']*100
last = last.loc[last.per>30]

In [578]:
len(osm), len(last)

(3102, 2144)

In [734]:
#osm = osm.loc[osm.osm_segment_id.isin(last.osm_segment_id.unique())]
m = kp.KeplerGl(height=600, data=dict(data=osm.loc[osm.osm_segment_id.isin(last.osm_segment_id.unique())], name='bus'))
m.add_data(data= lines_poly_10, name='lines')
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':          way_id  segment_seq           osm_segment_id  \
2      11105776            1  …

## 2. Approach with angles and perpendicularity

The idea of this approach is to see with which bus segment each osm segment intersects and compare their angles. 
If their angles are closer to perpendicularity than parallelism, then we'll discard those segments.

For this approach we first need to divide each bus segment in shorter lines of only two points. 
The reason behind this is that their original shape is more complex than a straight line and it is not posible to calculate their slope.

In [14]:
# Split bus segments into two-points segments and calculate it's angle
# For angle is atan(deltay/deltax)
# 180 degrees = pi 

import math

def small_segments(s):
    lat = s.coords.xy[1]
    lon = s.coords.xy[0]

    ss = [LineString([(lon[i], lat[i]), (lon[i+1], lat[i+1])]) for i in range(0,len(lat)-1)]
    angle = [math.atan2((lat[i+1] - lat[i]),(lon[i+1]-lon[i]))*180/math.pi for i in range(0,len(lat)-1)]
    ss_seq = list(range(0,len(ss)))
    n = len(ss)
    
    aux_df = pd.DataFrame()
    aux_df['ss_seq'] = pd.to_numeric([int(x) for x in ss_seq], downcast='signed')
    aux_df['angle'] = pd.to_numeric([int(x) for x in angle], downcast='signed')
    aux_df['geometry'] = ss
    
    return aux_df

def n_ss(s):
    n = len(s.coords.xy[1])-1
    return n    

ss = b_segments.loc[:, 'geometry'].apply(small_segments)
ss_df = pd.concat(list(ss))

n = b_segments.loc[:, 'geometry'].apply(n_ss)

seg_id = [[b_segments.loc[i, 'segment_id']]*n[i]  for i in range(0,len(n))]
seg_id = [item for sublist in seg_id for item in sublist]

ss_df['segment_id'] = pd.to_numeric([x for x in seg_id], downcast='signed')
ss_df['ss_id'] = list(range(1, len(seg_id)+1)) #ss_df['segment_id'].map(str) + '-' + ss_df['ss_seq'].map(str)
ss_gdf = gpd.GeoDataFrame(data=ss_df.drop('geometry', axis=1), geometry=ss_df.geometry)
ss_gdf.head()

,ss_seq,angle,segment_id,ss_id,geometry
0,0,2,1,1,"LINESTRING (-80.17219 25.92927, -80.17111 25.9..."
1,1,2,1,2,"LINESTRING (-80.17111 25.92932, -80.17067 25.9..."
2,2,-3,1,3,"LINESTRING (-80.17067 25.92934, -80.17054 25.9..."
3,3,4,1,4,"LINESTRING (-80.17054 25.92933, -80.17017 25.9..."
4,4,-93,1,5,"LINESTRING (-80.17017 25.92936, -80.17019 25.9..."


In [25]:
# Buffer the small bus segments
buffer_size = 7

ss_gdf.crs = {'init':'epsg:4326'}
ss_buffers = gpd.GeoDataFrame(data = ss_gdf.drop('geometry', axis=1), geometry = ss_gdf.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
ss_buffers.head()

,ss_seq,angle,segment_id,ss_id,geometry
0,0,2,1,1,"POLYGON ((-80.17112 25.92938, -80.17111 25.929..."
1,1,2,1,2,"POLYGON ((-80.17067 25.92940, -80.17067 25.929..."
2,2,-3,1,3,"POLYGON ((-80.17054 25.92940, -80.17053 25.929..."
3,3,4,1,4,"POLYGON ((-80.17018 25.92942, -80.17017 25.929..."
4,4,-93,1,5,"POLYGON ((-80.17012 25.92914, -80.17012 25.929..."


In [26]:
# Calculate angle of osm segments
def angle(s):
    lat = s.coords.xy[1]
    lon = s.coords.xy[0]
    
    angle = math.atan2((lat[1] - lat[0]),(lon[1]-lon[0]))*180/math.pi
    
    return angle

osm['osm_angle'] =  pd.to_numeric(osm.geometry.apply(angle).map(int), downcast='signed')

In [27]:
%%time
ss_intersections = gpd.sjoin(osm, ss_buffers, op='intersects').reset_index()
ss_intersections['diff'] =ss_intersections['angle']-ss_intersections['osm_angle']

# Keep the segments that have "similar" angles to their intersections
# We consider "similar" angles those that have similar values or almost 180 degrees different
# in the case that osm segment doesn't have a tag "oneway=yes"
diff = 40

parallel = ss_intersections.loc[(ss_intersections['diff'].map(abs)<diff)|(ss_intersections['diff'].map(abs)>(360-diff))]
oneway_seg = ss_intersections.loc[(ss_intersections['oneway']==0)&(ss_intersections['diff'].map(abs).between(180-diff,180+diff))]
osm_filtered = parallel.append(oneway_seg)

Wall time: 707 ms


In [33]:
m = kp.KeplerGl(height=600, data=dict(data=osm_filtered, name='bus'), #config=config
               )
m.add_data(data= ss_buffers, name='lines')
#m.add_data(data= ss_buffers.loc[~ss_buffers.ss_id.isin(osm_filtered.ss_id.unique())], name='matched lines')
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':        index                                           geometry     way_id  \
12       …

In [481]:
# intersections = gpd.sjoin(osm, buffers, op='intersects').reset_index()

### 2.1 Add the overlay approach

We still need to clean our OSM segments further. To do so, we will now apply the overlay approach.

This approach overlays two polygons and calculated the overlapping area. Then we match the segments with the biggest overlapping area.

To do this approach we need to buffer both, OSM segments and GTFS segments.

In [35]:
ss_buffers.head()

,ss_seq,angle,segment_id,ss_id,geometry
0,0,2,1,1,"POLYGON ((-80.17112 25.92938, -80.17111 25.929..."
1,1,2,1,2,"POLYGON ((-80.17067 25.92940, -80.17067 25.929..."
2,2,-3,1,3,"POLYGON ((-80.17054 25.92940, -80.17053 25.929..."
3,3,4,1,4,"POLYGON ((-80.17018 25.92942, -80.17017 25.929..."
4,4,-93,1,5,"POLYGON ((-80.17012 25.92914, -80.17012 25.929..."


In [37]:
buffer_size = 5

buffers_10 = gpd.GeoDataFrame(data = b_segments['segment_id'], geometry = b_segments.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
polygons = [p for p in buffers_10.geometry]
u = cascaded_union(polygons)

df = pd.DataFrame({'poly': [1], 'geometry':u.wkt})
df['geometry'] = df['geometry'].apply(wkt.loads)
lines_poly_10 = gpd.GeoDataFrame(data=df.poly, geometry = df.geometry)

osm_filtered.crs = {'init':'epsg:4326'}
osm_bu = gpd.GeoDataFrame(data = osm_filtered.drop('geometry', axis=1), geometry = osm_filtered.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
osm_bu['area_osm'] = osm_bu.area

poly_over = gpd.overlay(osm_bu.drop(['ss_seq', 'angle', 'segment_id', 'ss_id'], axis=1), lines_poly_10, how='intersection')
poly_over['area_overlay'] = poly_over.area
poly_over['per'] = poly_over['area_overlay']/poly_over['area_osm']*100
poly_over.head(1)

,index,way_id,oneway,segment_seq,osm_segment_id,osm_angle,index_right,diff,area_osm,poly,geometry,area_overlay,per
0,134,11128934,0,1,3305,1,3,1,8.195860e-08,1,"POLYGON ((-80.19784 25.88980, -80.19783 25.889...",7.554216e-08,92.171118


In [1020]:
# max_int = poly_over.pivot_table('per', index=['ss_id'], aggfunc='max').reset_index()
# max_int.rename(columns=dict(per='max_per'), inplace=True)
# poly_over = pd.merge(poly_over, max_int, how='left')
# poly_over = poly_over.loc[poly_over.per==poly_over.max_per]

39716 segments overlay
Reduced to 19816


In [39]:
m = kp.KeplerGl(height=600, data=dict(data=osm_filtered.loc[osm_filtered.osm_segment_id.isin(poly_over.osm_segment_id.unique())]), name='over',
               #config=config
               )
m.add_data(data= ss_gdf, name='lines')
m.add_data(data= poly_over, name='overlay')
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':        index                                           geometry     way_id  \
12       …

In [388]:
tlo = b_segments.loc[279, 'geometry'].coords.xy[0]
tla = b_segments.loc[279, 'geometry'].coords.xy[1]

tpo = [Point(tlo[i], tla[i]) for i in range(0, len(tlo))]

tli = [LineString([tpo[i], tpo[i+1]]) for i in range(0,len(tpo)-1)]

In [442]:
t_ = list(range(0,len(tli)))
t = pd.DataFrame()
t['ss_id'] = t_
t['segment_id'] = ['8908-8826']*len(tli)
t = gpd.GeoDataFrame(data=t, geometry=tli)

In [461]:
buffer_size = 3

t.crs = {'init':'epsg:4326'}

tbu = gpd.GeoDataFrame(data = t.drop('geometry', axis=1), geometry = t.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
tbu

,ss_id,segment_id,geometry
0,0,8908-8826,"POLYGON ((-80.19704 25.88977, -80.19705 25.889..."
1,1,8908-8826,"POLYGON ((-80.19782 25.88974, -80.19782 25.889..."
2,2,8908-8826,"POLYGON ((-80.19865 25.88970, -80.19866 25.889..."
3,3,8908-8826,"POLYGON ((-80.19861 25.88955, -80.19861 25.889..."
4,4,8908-8826,"POLYGON ((-80.19860 25.88902, -80.19860 25.889..."
5,5,8908-8826,"POLYGON ((-80.19863 25.88899, -80.19863 25.888..."
6,6,8908-8826,"POLYGON ((-80.19858 25.88834, -80.19858 25.888..."


In [457]:
#osm_test = intersections.loc[intersections.segment_id=='8908-8826']
osm.crs = {'init':'epsg:4326'}
osm_bu = gpd.GeoDataFrame(data = osm.drop('geometry', axis=1), geometry = osm.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
osm_bu['area_osm'] = osm_bu.area
osm_bu

,way_id,segment_seq,osm_segment_id,geometry,area_osm
0,11103228,7,98975025 - 98975028,"POLYGON ((-80.19708 25.88978, -80.19708 25.889...",4.530003e-08
1,11103228,8,98975028 - 98975031,"POLYGON ((-80.19717 25.89181, -80.19717 25.891...",1.242749e-07
2,11105776,1,98999741 - 2915146657,"POLYGON ((-80.14509 25.96005, -80.14509 25.960...",1.277068e-08
3,11106364,1,6366190747 - 6366190722,"POLYGON ((-80.14519 25.95936, -80.14519 25.959...",8.363258e-09
4,11107093,1,99010535 - 99010537,"POLYGON ((-80.19108 25.83185, -80.19108 25.831...",6.252647e-08
...,...,...,...,...,...
2759,835882554,2,7187981992 - 6945559259,"POLYGON ((-80.17005 25.92674, -80.17005 25.926...",2.200858e-08
2760,835882554,3,6945559259 - 99064510,"POLYGON ((-80.17004 25.92663, -80.17004 25.926...",9.428639e-09
2761,835882555,1,99189930 - 7802016538,"POLYGON ((-80.17007 25.92719, -80.17007 25.927...",2.043142e-08
2762,835882556,1,7802016539 - 6945095202,"POLYGON ((-80.17000 25.92584, -80.17000 25.925...",2.486940e-08


In [486]:
max_int = over.pivot_table('per', index='ss_id', aggfunc='max').reset_index()
max_int.rename(columns=dict(per='max_per'), inplace=True)
over = pd.merge(over, max_int, how='left')
over = over.loc[over.per==over.max_per]
over

,ss_id,segment_id,way_id,segment_seq,osm_segment_id,area_osm,geometry,area_overlay,per,max_per
3,1,8908-8826,11128934,2,99139906 - 98975028,4.511272e-08,"POLYGON ((-80.19782 25.88974, -80.19782 25.889...",3.373310e-08,74.775140,74.775140
5,0,8908-8826,11128934,3,98975028 - 99139908,4.479705e-08,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",2.983087e-08,66.591157,66.591157
10,2,8908-8826,11128934,1,99139904 - 99139906,4.747895e-08,"POLYGON ((-80.19866 25.88970, -80.19866 25.889...",4.126942e-08,86.921505,86.921505
15,3,8908-8826,105564714,4,99123753 - 99139904,4.528520e-08,"POLYGON ((-80.19863 25.88952, -80.19863 25.889...",1.027158e-08,22.681991,22.681991
16,4,8908-8826,105564714,4,99123753 - 99139904,4.528520e-08,"POLYGON ((-80.19865 25.88900, -80.19866 25.889...",2.909899e-08,64.257179,64.257179
24,5,8908-8826,105564714,3,99321452 - 99123753,4.297106e-08,"POLYGON ((-80.19863 25.88899, -80.19863 25.888...",2.584099e-09,6.013579,6.013579
25,6,8908-8826,105564714,3,99321452 - 99123753,4.297106e-08,"POLYGON ((-80.19860 25.88832, -80.19860 25.888...",4.099219e-08,95.394875,95.394875


In [496]:
over.drop_duplicates(subset=['segment_id', 'osm_segment_id'])

,ss_id,segment_id,way_id,segment_seq,osm_segment_id,area_osm,geometry,area_overlay,per,max_per
3,1,8908-8826,11128934,2,99139906 - 98975028,4.511272e-08,"POLYGON ((-80.19782 25.88974, -80.19782 25.889...",3.373310e-08,74.775140,74.775140
5,0,8908-8826,11128934,3,98975028 - 99139908,4.479705e-08,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",2.983087e-08,66.591157,66.591157
10,2,8908-8826,11128934,1,99139904 - 99139906,4.747895e-08,"POLYGON ((-80.19866 25.88970, -80.19866 25.889...",4.126942e-08,86.921505,86.921505
15,3,8908-8826,105564714,4,99123753 - 99139904,4.528520e-08,"POLYGON ((-80.19863 25.88952, -80.19863 25.889...",1.027158e-08,22.681991,22.681991
24,5,8908-8826,105564714,3,99321452 - 99123753,4.297106e-08,"POLYGON ((-80.19863 25.88899, -80.19863 25.888...",2.584099e-09,6.013579,6.013579


In [489]:
kp.KeplerGl(data=dict(data=osm_bu.loc[osm_bu.osm_segment_id=='99321452 - 99123753'], name='overlay'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':         way_id  segment_seq       osm_segment_id  \
772  105564714            3  993214…

In [480]:
kp.KeplerGl(data=dict(data=osm_bu.loc[osm.osm_segment_id.isin(over.osm_segment_id.unique())], name='overlay'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':         way_id  segment_seq       osm_segment_id  \
115   11128934            1  991399…

In [466]:
kp.KeplerGl(data=dict(data=over, name='overlay'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':     ss_id segment_id     way_id  segment_seq       osm_segment_id  \
0       0  8908-88…

In [464]:
kp.KeplerGl(data=dict(data=osm.loc[osm.osm_segment_id.isin(over.osm_segment_id.unique())], name='overlay'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':          way_id  segment_seq       osm_segment_id  \
0      11103228            7  9897…

In [463]:
kp.KeplerGl(data=dict(data=osm.loc[osm.], name='overlay'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':    ss_id segment_id                                           geometry
0      0  8908-8…

In [487]:
kp.KeplerGl(data=dict(data=t, name='bus'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':    ss_id segment_id                                           geometry
0      0  8908-8…

In [495]:
t.loc[5].geometry.coords.xy

(array('d', [-80.198631, -80.19863199999999]),
 array('d', [25.889015, 25.889015]))

In [412]:
buffer_size = 3

t.crs = {'init':'epsg:4326'}

tbu = gpd.GeoDataFrame(data = t['t'], geometry = t.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
kp.KeplerGl(height=400, data=dict(data=tbu, name='buffers'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':    t                                           geometry
0  0  POLYGON ((-80.19704 25.88…

In [377]:
tbu

,t,geometry
0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889..."
1,t,"POLYGON ((-80.19782 25.88974, -80.19782 25.889..."
2,t,"POLYGON ((-80.19865 25.88970, -80.19866 25.889..."
3,t,"POLYGON ((-80.19861 25.88955, -80.19861 25.889..."
4,t,"POLYGON ((-80.19860 25.88902, -80.19860 25.889..."
5,t,"POLYGON ((-80.19863 25.88899, -80.19863 25.888..."
6,t,"POLYGON ((-80.19858 25.88834, -80.19858 25.888..."


In [376]:
osm_test = intersections.loc[intersections.segment_id=='8908-8826']
osm_test.crs = {'init':'epsg:4326'}
osm_bu = gpd.GeoDataFrame(data = osm_test.drop('geometry', axis=1), geometry = osm_test.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
osm_bu

,index,way_id,segment_seq,osm_segment_id,index_right,segment_id,geometry
0,0,11103228,7,98975025 - 98975028,279,8908-8826,"POLYGON ((-80.19708 25.88978, -80.19708 25.889..."
1,1,11103228,8,98975028 - 98975031,279,8908-8826,"POLYGON ((-80.19717 25.89181, -80.19717 25.891..."
2,94,11125963,1,99123753 - 98975025,279,8908-8826,"POLYGON ((-80.19701 25.88910, -80.19701 25.889..."
3,115,11128934,1,99139904 - 99139906,279,8908-8826,"POLYGON ((-80.19784 25.88978, -80.19783 25.889..."
4,116,11128934,2,99139906 - 98975028,279,8908-8826,"POLYGON ((-80.19705 25.88981, -80.19705 25.889..."
5,117,11128934,3,98975028 - 99139908,279,8908-8826,"POLYGON ((-80.19628 25.88987, -80.19627 25.889..."
6,207,11141768,1,99139904 - 99221219,279,8908-8826,"POLYGON ((-80.19920 25.88968, -80.19920 25.889..."
7,323,11152057,1,99139906 - 99311970,279,8908-8826,"POLYGON ((-80.19794 25.89178, -80.19794 25.891..."
8,556,11169671,1,99321452 - 99312001,279,8908-8826,"POLYGON ((-80.19774 25.88839, -80.19774 25.888..."
9,771,105564714,2,99091949 - 99321452,279,8908-8826,"POLYGON ((-80.19864 25.88834, -80.19864 25.888..."


In [397]:
t_int = gpd.sjoin(osm_bu.drop(['index','index_right'], axis=1), tbu.loc[0:1,:], op='intersects').reset_index()
t_int

,index,way_id,segment_seq,osm_segment_id,segment_id,geometry,index_right,t
0,0,11103228,7,98975025 - 98975028,8908-8826,"POLYGON ((-80.19708 25.88978, -80.19708 25.889...",0,t
1,1,11103228,8,98975028 - 98975031,8908-8826,"POLYGON ((-80.19717 25.89181, -80.19717 25.891...",0,t
2,4,11128934,2,99139906 - 98975028,8908-8826,"POLYGON ((-80.19705 25.88981, -80.19705 25.889...",0,t
3,5,11128934,3,98975028 - 99139908,8908-8826,"POLYGON ((-80.19628 25.88987, -80.19627 25.889...",0,t
4,0,11103228,7,98975025 - 98975028,8908-8826,"POLYGON ((-80.19708 25.88978, -80.19708 25.889...",1,t
5,1,11103228,8,98975028 - 98975031,8908-8826,"POLYGON ((-80.19717 25.89181, -80.19717 25.891...",1,t
6,3,11128934,1,99139904 - 99139906,8908-8826,"POLYGON ((-80.19784 25.88978, -80.19783 25.889...",1,t
7,4,11128934,2,99139906 - 98975028,8908-8826,"POLYGON ((-80.19705 25.88981, -80.19705 25.889...",1,t
8,5,11128934,3,98975028 - 99139908,8908-8826,"POLYGON ((-80.19628 25.88987, -80.19627 25.889...",1,t
9,7,11152057,1,99139906 - 99311970,8908-8826,"POLYGON ((-80.19794 25.89178, -80.19794 25.891...",1,t


In [417]:
type(tbu.head(1))

geopandas.geodataframe.GeoDataFrame

In [418]:
kp.KeplerGl(height=400, data=dict(data=tbu.head(1), name='buffers'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':    t                                           geometry
0  0  POLYGON ((-80.19704 25.88…

In [410]:
kp.KeplerGl(height=400, data=dict(data=t_int.loc[t_int.index_right==0], name='buffers'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':    index    way_id  segment_seq       osm_segment_id segment_id  \
0      0  11103228  …

In [406]:
gpd.overlay(tbu.head(1), t_int, how='intersection')

,t_1,index,way_id,segment_seq,osm_segment_id,segment_id,index_right,t_2,geometry
0,t,4,11128934,2,99139906 - 98975028,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889..."
1,t,4,11128934,2,99139906 - 98975028,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889..."
2,t,1,11103228,8,98975028 - 98975031,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889..."
3,t,1,11103228,8,98975028 - 98975031,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889..."
4,t,0,11103228,7,98975025 - 98975028,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889..."
5,t,0,11103228,7,98975025 - 98975028,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889..."
6,t,5,11128934,3,98975028 - 99139908,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889..."
7,t,5,11128934,3,98975028 - 99139908,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889..."


In [408]:
kp.KeplerGl(height=400, data=dict(data=gpd.overlay(tbu.head(1), t_int, how='intersection')))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':   t_1  index    way_id  segment_seq       osm_segment_id segment_id  \
0   t      4  11…

In [428]:
tbu.head(1).area.values[0]

3.3146897501504983e-08

In [429]:
gpd.overlay(tbu.head(1), t_int, how='intersection').area.values/tbu.head(1).area.values[0]*100

array([ 5.34658962,  5.34658962,  7.44132152,  7.44132152,  5.28224379,
        5.28224379, 89.99597842, 89.99597842])

In [430]:
ov = gpd.overlay(tbu.head(1), t_int, how='intersection')
ov['area'] = ov.area
ov

,t_1,index,way_id,segment_seq,osm_segment_id,segment_id,index_right,t_2,geometry,area
0,0,4,11128934,2,99139906 - 98975028,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",1.772229e-09
1,0,4,11128934,2,99139906 - 98975028,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",1.772229e-09
2,0,1,11103228,8,98975028 - 98975031,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",2.466567e-09
3,0,1,11103228,8,98975028 - 98975031,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",2.466567e-09
4,0,0,11103228,7,98975025 - 98975028,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",1.750900e-09
5,0,0,11103228,7,98975025 - 98975028,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",1.750900e-09
6,0,5,11128934,3,98975028 - 99139908,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",2.983087e-08
7,0,5,11128934,3,98975028 - 99139908,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",2.983087e-08


In [431]:
t_int['area_t'] = t_int.area
t_int

,index,way_id,segment_seq,osm_segment_id,segment_id,geometry,index_right,t,area_t
0,0,11103228,7,98975025 - 98975028,8908-8826,"POLYGON ((-80.19708 25.88978, -80.19708 25.889...",0,t,4.530003e-08
1,1,11103228,8,98975028 - 98975031,8908-8826,"POLYGON ((-80.19717 25.89181, -80.19717 25.891...",0,t,1.242749e-07
2,4,11128934,2,99139906 - 98975028,8908-8826,"POLYGON ((-80.19705 25.88981, -80.19705 25.889...",0,t,4.511272e-08
3,5,11128934,3,98975028 - 99139908,8908-8826,"POLYGON ((-80.19628 25.88987, -80.19627 25.889...",0,t,4.479705e-08
4,0,11103228,7,98975025 - 98975028,8908-8826,"POLYGON ((-80.19708 25.88978, -80.19708 25.889...",1,t,4.530003e-08
5,1,11103228,8,98975028 - 98975031,8908-8826,"POLYGON ((-80.19717 25.89181, -80.19717 25.891...",1,t,1.242749e-07
6,3,11128934,1,99139904 - 99139906,8908-8826,"POLYGON ((-80.19784 25.88978, -80.19783 25.889...",1,t,4.747895e-08
7,4,11128934,2,99139906 - 98975028,8908-8826,"POLYGON ((-80.19705 25.88981, -80.19705 25.889...",1,t,4.511272e-08
8,5,11128934,3,98975028 - 99139908,8908-8826,"POLYGON ((-80.19628 25.88987, -80.19627 25.889...",1,t,4.479705e-08
9,7,11152057,1,99139906 - 99311970,8908-8826,"POLYGON ((-80.19794 25.89178, -80.19794 25.891...",1,t,1.243119e-07


In [434]:
con = pd.merge(ov, t_int[['osm_segment_id', 'area_t']].drop_duplicates(subset='osm_segment_id'), left_on='osm_segment_id', right_on='osm_segment_id', how='left')

In [435]:
con['perc'] = con['area']/con['area_t']*100

In [436]:
con

,t_1,index,way_id,segment_seq,osm_segment_id,segment_id,index_right,t_2,geometry,area,area_t,perc
0,0,4,11128934,2,99139906 - 98975028,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",1.772229e-09,4.511272e-08,3.928445
1,0,4,11128934,2,99139906 - 98975028,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",1.772229e-09,4.511272e-08,3.928445
2,0,1,11103228,8,98975028 - 98975031,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",2.466567e-09,1.242749e-07,1.984767
3,0,1,11103228,8,98975028 - 98975031,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",2.466567e-09,1.242749e-07,1.984767
4,0,0,11103228,7,98975025 - 98975028,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",1.750900e-09,4.530003e-08,3.865118
5,0,0,11103228,7,98975025 - 98975028,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",1.750900e-09,4.530003e-08,3.865118
6,0,5,11128934,3,98975028 - 99139908,8908-8826,0,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",2.983087e-08,4.479705e-08,66.591157
7,0,5,11128934,3,98975028 - 99139908,8908-8826,1,t,"POLYGON ((-80.19704 25.88977, -80.19705 25.889...",2.983087e-08,4.479705e-08,66.591157


In [437]:
osm_test.loc[osm_test.osm_segment_id=='98975028 - 99139908']

,index,way_id,segment_seq,osm_segment_id,geometry,index_right,segment_id
5,117,11128934,3,98975028 - 99139908,"LINESTRING (-80.19705 25.88978, -80.19627 25.8...",279,8908-8826


In [438]:
kp.KeplerGl(height=400, data=dict(data=osm_test.loc[osm_test.osm_segment_id=='98975028 - 99139908']))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':    index    way_id  segment_seq       osm_segment_id  \
5    117  11128934            3…

# OSM segments

In [75]:
!mdt create-geometry-file miami 2019 --bbox="-80.198654,25.773262,-80.141864,25.966199"  > miami_osm.geojson

'mdt' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# We get an xml with the API and then have to parse it.
# Overpass API
# 'https://overpass-api.de/api/map?bbox=-80.19866398,25.77325297,-80.14185402,25.96620802'

import xml.etree.ElementTree as ET
tree = ET.parse(r"C:\Users\santi\Dropbox (Remix)\Remix\Data Team\Data Analytics\Corridor Analysis\osm_segments\map (2)")
root = tree.getroot()

ids = []
lat = []
lon = []

for node in root.findall('node'):
    ids = ids + [node.attrib['id']]
    lat = lat + [node.attrib['lat']]
    lon = lon + [node.attrib['lon']]

nodes = pd.DataFrame()
nodes['node_id'] = ids
nodes['lat'] = lat
nodes['lon'] = lon

nodes['geometry'] = [Point(float(row.lon), float(row.lat)) for index, row in nodes.iterrows()]
n = gpd.GeoDataFrame(data=nodes.drop('geometry', axis=1), geometry =nodes.geometry )

In [ ]:
kp.KeplerGl(data=dict(data=n, name='nodes'))

In [ ]:
ways_list = []
node_sequence = []

for way in root.findall('way'):
    ways_list = ways_list + [way.attrib['id']]*len(way.findall('nd'))
    node_sequence = node_sequence + list(range(1, len(way.findall('nd'))+1))
    
nodes_list = [nd.attrib['ref'] for nd in root.iter('nd')]

ways = pd.DataFrame()
ways['way_id'] = ways_list
ways['node_id'] = nodes_list
ways['node_sequence'] = node_sequence

In [ ]:
ways_list = []
ways_tagsk = []
ways_tagsv = []

for way in root.findall('way'):
    ways_list = ways_list + [way.attrib['id']]*len(way.findall('tag'))
    
    for tag in way.findall('tag'):
        ways_tagsk = ways_tagsk + [tag.attrib['k']]
        ways_tagsv = ways_tagsv + [tag.attrib['v']]
        
tags = pd.DataFrame()
tags['way_id'] = ways_list
tags['k'] = ways_tagsk
tags['v'] = ways_tagsv 

tag_filter = ['motorway', 'motorway_link','trunk' , 'trunk_link','primary' , 'primary_link','secondary' , 
              'secondary_link','tertiary' , 'tertiary_link','unclassified' , 
              'unclassified_link','residential' , 'residential_link','service' , 'service_link','living_street']

way_filter = tags.loc[(tags.v.isin(tag_filter))&(tags.k=='highway'), 'way_id'].unique()

ways = ways.loc[ways.way_id.isin(way_filter)]

In [89]:
segments = pd.merge(ways, nodes, how='left')

from shapely.geometry import Point, LineString

segments['end_node_id'] = segments['node_id'].shift(-1)
segments['next_way_id'] = segments['way_id'].shift(-1)
segments['next_geometry'] = segments['geometry'].shift(-1)
segments = segments.loc[~segments.next_geometry.isnull()]
segments['seg_geometry'] = segments.apply(lambda x: LineString([x.geometry, x.next_geometry]), axis=1)
segments['keep'] = segments['way_id'] ==segments['next_way_id']
segments = segments.loc[segments['keep'], ['way_id', 'node_id', 'end_node_id', 'seg_geometry']]
segments.columns=['way_id', 'start_node_id', 'end_node_id', 'geometry']
segments['segment_id'] = segments.start_node_id + '-' + segments.end_node_id
segments_gdf = gpd.GeoDataFrame(data=segments.loc[:,['segment_id','way_id', 'start_node_id', 'end_node_id']],
                               geometry=segments.geometry)

In [90]:
kp.KeplerGl(data=dict(data=segments_gdf, name='segments'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':                 segment_id     way_id start_node_id end_node_id  \
0      98975064-5435…

# Get the geometry of OSM segments

After exploring different alternatives it seems that there are no good ways of getting both the geometry and the start and end nodes of each segment.

In contrast, we can manage to get the geometry in one way and the node in a different one.
To get the geometries I went to this [link](https://overpass-turbo.eu/) and ran a script with the bounding box of the coordinates surrounding the area I need.

That returns a Geojson with the geometry for each segment. A sample script can be found below


`(way(51.477,-0.001,51.478,0.001);
  node(w);
  relation(51.477,-0.001,51.478,0.001); );
out skel;`

In [144]:
e = gpd.read_file(r"C:\Users\santi\Dropbox (Remix)\Remix\Data Team\Data Analytics\Corridor Analysis\osm_segments\export (1).geojson")
m = kp.KeplerGl(data=dict(data=e.loc[0:1,:], name='example'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':             id          @id @relations  \
0  way/8879404  way/8879404       None   
1  …

In [238]:
print(points[0])

POINT (-0.0021208 51.4812272)


In [153]:
from shapely.geometry import Point


points = [Point(c) for c in e.loc[0,'geometry'].coords]
df = pd.DataFrame()
df['node'] = range(0,len(points))

m = kp.KeplerGl(data=dict(data=gpd.GeoDataFrame(df, geometry = points), name='nodes'))
m.add_data(data=e.loc[0:1,:], name='segments')
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':     node                   geometry
0      0  POINT (-0.00212 51.48123)
1      1  POINT…

In [ ]:
# Overpass API
'https://overpass-api.de/api/map?bbox=-80.3313,25.7084,-79.7923,25.9658'